In [36]:
import csv
import cv2
import numpy as np

lines=[]
with open('C:\data\driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
images = []
measurments = []
for line in lines:
    image_center = cv2.imread(line[0])
    image_left = cv2.imread(line[1])
    image_right = cv2.imread(line[2])
    images.extend([image, image_left, image_right])
    steering = float(line[3])
    measurments.extend([steering, steering+0.1, steering-0.1])
    
X_train = np.array(images)
y_train = np.array(measurments)

print("features Shape: ", X_train.shape)
print("lables Shape: ", len(y_train))

features Shape:  (3090, 160, 320, 3)
lables Shape:  3090


In [37]:
#Augmentation - flip all images

aug_images, aug_measurments= [], []
for image, measurment in zip(images, measurments):
    aug_images.append(cv2.flip(image,1))
    aug_measurments.append(measurment*-1.0)
    

print("augmented features Shape: ", len(aug_images))
print("augmented lables Shape: ", len(aug_measurments))

augmented features Shape:  3090
augmented lables Shape:  3090


In [ ]:
X_train = np.concatenate([X_train, aug_images])
y_train = np.concatenate([y_train, aug_measurments])
print(X_train.shape, y_train.shape)

(6180, 160, 320, 3) (6180,)


In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.layers.convolutional import Conv2D, MaxPooling2D

model = Sequential()
#normalization and mean zero
model.add(Lambda(lambda x: x/255.0 -0.5, input_shape=(160,320,3))) 
#cropping2D layer
model.add(Cropping2D(cropping=((70,25), (0,0)), input_shape=(160,320,3)))
#model.add(Flatten())
model.add(Dense(1))
model.add(Conv2D(6, kernel_size=(5, 5), padding='valid', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(6, kernel_size=(5, 5), padding='valid', activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=5)

model.save('model.h5')

C:\ProgramData\Anaconda3\lib\site-packages\keras\models.py:846: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 4944 samples, validate on 1236 samples
Epoch 1/5
4944/4944 [==============================] - 242s - loss: 0.0867 - val_loss: 0.0221
Epoch 2/5
4512/4944 [==========================>...] - ETA: 20s - loss: 0.0105